In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [2]:
import { TextLoader } from "langchain/document_loaders/fs/text";

const loader = new TextLoader("./data/qiu.txt");
const docs = await loader.load();

In [3]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 10,
  });

const splitDocs = await splitter.splitDocuments(docs);

In [4]:
import { AlibabaTongyiEmbeddings } from "@langchain/community/embeddings/alibaba_tongyi";

const embeddings = new AlibabaTongyiEmbeddings({});

In [5]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

In [6]:
const retriever = vectorstore.asRetriever(2);

In [7]:
const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍宏原子假设的理论");
console.log(res);

[
  Document {
    pageContent: "“对于宏电子来说，波粒二象性中波的形态占很大比重，所以它的大小的意义与我们常识中的完全不同。它还有很多令人难以置信的特性，我们以后会慢慢看到的，我相信这会改变大家对世界的看法。不过现在，我们要先给这些"... 334 more characters,
    metadata: {
      source: "./data/qiu.txt",
      loc: { lines: { from: 1464, to: 1471 } }
    }
  },
  Document {
    pageContent: "“如果人类生活在一个没有摩擦力的世界，牛顿三定律可能会在更早的时候由更普通的人来发现。当你本身已经成为一个量子态的宏粒子，理解那个世界自然比我们要容易得多。”\n" +
      "　　于是，基地开始了捕获宏原子核的工作。"... 167 more characters,
    metadata: {
      source: "./data/qiu.txt",
      loc: { lines: { from: 2222, to: 2224 } }
    }
  }
]


In [8]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (list) => {
return list.map((document) => document.pageContent).join("\n")
};

const contextRetriverChain = RunnableSequence.from([
(input) => input.question,
retriever,
convertDocsToString
]);

In [9]:
const result = await contextRetriverChain.invoke({ question: "原文中，谁提出了宏原子的假设？并详细介绍宏原子假设的理论"});

console.log(result);

“对于宏电子来说，波粒二象性中波的形态占很大比重，所以它的大小的意义与我们常识中的完全不同。它还有很多令人难以置信的特性，我们以后会慢慢看到的，我相信这会改变大家对世界的看法。不过现在，我们要先给这些大电子取一个名字，它们是宏观尺度的电子，就叫宏电子吧。”
　　“那么，像刚才说的，是否存在宏质子和宏中子呢？”
　　“应该存在，不过由于它们不能被激发，我们很难发现它们。”
　　“丁教授，你的梦实现了。”林云说，除了丁仪和我，别的人还不太明白她这话的意思。
　　“是啊是啊，真有西瓜这么大的基本粒子摆上物理学家的桌面了，下一步我们肯定要研究它们的内部结构，那也是由弯曲的空间构成的结构，虽然也很难，但我相信比研究微观粒子的结构不知要容易多少倍。”
　　“那也存在宏原子了？三种宏粒子应该是能够组成原子的啊！”
　　“是的，应该有宏原子。”
　　“我们所捕获到的那个空泡，哦，那个宏电子，是自由电子呢，还是一个宏原子中的电子？如果是后者，那这个宏原子的原子核在哪里呢？”
“如果人类生活在一个没有摩擦力的世界，牛顿三定律可能会在更早的时候由更普通的人来发现。当你本身已经成为一个量子态的宏粒子，理解那个世界自然比我们要容易得多。”
　　于是，基地开始了捕获宏原子核的工作。
　　首先，用空泡光学探测系统精确观测宏电子在空间中的自由运行状态，现在知道，宏电子或它被激发后形成的球状闪电那轨迹复杂的飘行，实际上是一种不断的量子跃迁，但在我们的视觉中它的运行是连续的。运用张彬墓碑上出现的那个伟大的数学模型，通过对这种跃迁运动各种参数的复杂计算，就能够确定宏原子核的位置，如果这个宏电子确实是属于某个宏原子的话。


In [10]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE = `
    你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
    并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，
    
    以下是原文中跟用户回答相关的内容：
    {context}
    
    现在，你需要基于原文，回答以下问题：
    {question}`;

const prompt = ChatPromptTemplate.fromTemplate(
    TEMPLATE
);

In [11]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
const model = new ChatAlibabaTongyi({
  model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
  temperature: 1,
});

In [12]:
import { StringOutputParser } from "@langchain/core/output_parsers";
  
const ragChain = RunnableSequence.from([
    {
        context: contextRetriverChain,
        question: (input) => input.question,
    },
    prompt,
    model,
    new StringOutputParser()
]);

In [13]:
const res = await ragChain.invoke({
    question: "什么是球状闪电"
});  
console.log(res);

球状闪电是一种在《球状闪电》这部科幻小说中描述的自然现象，它并非严格意义上的科学术语，而是刘慈欣作品中的虚构概念。在书中，球状闪电被描绘为一种能产生量子态、具有神秘特性的能量体，它可以被技术手段如雷球机枪操控。尽管它的军事应用被提及，但其本质和工作原理并未详尽揭示，仅仅是宏电子技术让人类得以窥探物质微观世界的另一种方式。


In [14]:
const res = await ragChain.invoke({
    question: "静夜思这首诗是什么"
});  
console.log(res);

原文中并没有提到“静夜思”这首诗，所以无法提供相关的内容。
